In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import StrainerStatus
from benchmark_consts import *

In [2]:
qs = UNSTABLE_VERUS
options = DebugOptions()
options.is_verus = True

mv = BenchViewer(qs, options)

[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [init] 8d058577cd trace slow unknown, fallback to timeout 


In [3]:
mv.status.print_status()

| category                |   count | percentage   |
|-------------------------|---------|--------------|
| StrainerStatus.FINISHED |      67 | 95.71 %      |
| StrainerStatus.NO_PROOF |       3 | 4.29 %       |
| total                   |      70 | 100.00 %     |


In [4]:
discount = [
    ("08719204aa", "4b9e624e"),
    ("81b159a977", "fbe192f3"),
    ("2556504d15", "7ab783b0"),
    ("6540edb885", "5e546c11"),
    ("82449bd2e8", "4c612f9b"),
    ("a1b4b3fd4a", "4e444e51"),
    ("55b4af82a4", "02d50371"),
    ("2556504d15", "0e424555"),
    ("4d2d4fbc20", "5a3b4208"),
    ("82449bd2e8", "905f3174"),
    ("2045867a58", "7ec12ea5"),
    ("55b4af82a4", "781157b6"),
    ("68ac429152", "75d801e3"),
]

In [5]:
def fixable_after_discount(dbg):
    report =dbg.report
    name_hash = dbg.name_hash
    fixable = False
    for edit_path in report.stabilized['edit_path']:
        edit_hash = os.path.basename(edit_path).split('.')[0]
        if (name_hash, edit_hash) in discount:
            continue
        fixable = True
    return fixable

In [6]:
from utils.analysis_utils import fmt_percent

bench_total = len(mv.status.tally)
bench_finished = len(mv.status[StrainerStatus.FINISHED])
# to_fixable = 0
to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in mv.status[StrainerStatus.FINISHED]:
    dbg = mv[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases[q] = q in mv.fixable and fixable_after_discount(dbg)
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases[q] = q in mv.fixable and fixable_after_discount(dbg)
    else:
        print("Unexpected mode", dbg.mode)
        assert False

to_fixable = sum(to_cases.values())
ff_fixable = sum(ff_cases.values())

print("Test Finished:", print_break_down(bench_finished, bench_total))
print("\tTOs:", print_break_down(len(to_cases), bench_finished))
print("\t\tTO fixable:", print_break_down(to_fixable, len(to_cases)))
print("\tFFs:", print_break_down(len(ff_cases), bench_finished))
print("\t\tFF fixable:", print_break_down(ff_fixable, len(ff_cases)))

print("Test Not Finished:", print_break_down(len(mv.status[StrainerStatus.NOT_CREATED]), bench_total))
print("No Proofs:", print_break_down(len(mv.status[StrainerStatus.NO_PROOF]), bench_total))
print("* fixable:", print_break_down(to_fixable + ff_fixable, bench_total))

Test Finished: 67/70 (95.7%)
	TOs: 38/67 (56.7%)
		TO fixable: 25/38 (65.8%)
	FFs: 29/67 (43.3%)
		FF fixable: 22/29 (75.9%)
Test Not Finished: 0/70 (0.0%)
No Proofs: 3/70 (4.3%)
* fixable: 47/70 (67.1%)


In [7]:
for query in mv.fixable:
    report = mv[query].report
    name_hash = mv[query].name_hash

    for edit_path in report.stabilized['edit_path']:
        edit_query = os.path.basename(edit_path)
        bundle_path = f"data/projs/verus_singleton_bundle/base.z3/{name_hash}.{edit_query}"

In [8]:
for q in mv.status[StrainerStatus.NO_PROOF]:
    print(q)

data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2
data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2
data/projs/verismo.dedup/base.z3/arch__ptram__ptram_p2.smt2


In [9]:
for q in mv.status[StrainerStatus.NOT_CREATED]:
    print("./src/debugger3.py -m auto -i", mv[q].name_hash, "--create-project")

In [10]:
from utils.system_utils import list_smt2_files

for q in mv.status[StrainerStatus.UNFILTERED]:
    assert len(list_smt2_files(mv[q].strainer.filter_dir)) != 0
    print("./src/make_spaghet.py --verus -i", mv[q].strainer.filter_dir)

for q in mv.status[StrainerStatus.NOT_TESTED]:
    assert len(list_smt2_files(mv[q].strainer.test_dir)) != 0
    print("./src/make_spaghet.py --verus -i", mv[q].strainer.test_dir)

In [11]:
# from utils.cluster_utils import get_sync_commands
# from utils.system_utils import write_misc_script

# for q in mv.status.tally:
#     dbg = mv[q]
#     dbg.collect_garbage()

# write_misc_script("send.sh", get_sync_commands("g2001", mv.get_sync_dirs()))

In [15]:
from debugger.edit_info import EditAction, EditInfo

VERUS_SKOLEM_DIR = "data/projs/verus.skolem/base.z3"

skolem_failed = []

for q in mv.unfixable:
    if q not in ff_cases:
        continue
    dbg = mv[q]
    qname = dbg.editor.choose_qanme_to_skolemize()
    if qname is None:
        skolem_failed.append(q)
        continue
    ei = EditInfo(VERUS_SKOLEM_DIR, {qname: EditAction.SKOLEMIZE})
    edit_hash = ei.get_id()
    name_hash = dbg.name_hash

    if not dbg.editor.edit_by_qname(qname, EditAction.SKOLEMIZE):
        skolem_failed.append(q)
        continue
        
    query_path = f"{VERUS_SKOLEM_DIR}/{name_hash}.{edit_hash}.smt2"

    if not dbg.editor.save(query_path):
        skolem_failed.append(q)
        continue


[DEBUG] [skolem] i$!skolem_user_vstd__seq__axiom_seq_ext_equal_14!0 -> skv_6e6d8de87cb115f5_1 
[DEBUG] [edit] proof path: dbg/d057fff3f5/proofs/rename.17608056682826294385.proof 
[DEBUG] [edit] trace path: dbg/d057fff3f5/traces/shuffle.227536422975727424 
[DEBUG] [skolem] is in the goal! 
[DEBUG] [skolem] i$!skolem_user_lib__spec__cyclicbuffer__log_entry_alive_wrap_around_51!1 -> skv_7e8cb799170a4a47_1 


[INFO] [edit] [skolem] user_vstd__seq__axiom_seq_ext_equal_14 creates 1 skolem funs, 1 asserts 
(assert (=> (fuel_bool fuel%vstd!seq.axiom_seq_ext_equal.) (forall ((A&. Dcr) (A& Type) (s1! Poly) (s2! Poly)) (! (=> (and (has_type s1! (TYPE%vstd!seq.Seq. A&. A&)) (has_type s2! (TYPE%vstd!seq.Seq. A&. A&))) (= (ext_eq false (TYPE%vstd!seq.Seq. A&. A&) s1! s2!) (and (= (vstd!seq.Seq.len.? A&. A& s1!) (vstd!seq.Seq.len.? A&. A& s2!)) (forall ((i$ Poly)) (! (=> (has_type i$ INT) (=> (and (<= 0 (%I i$)) (< (%I i$) (vstd!seq.Seq.len.? A&. A& s1!))) (= (vstd!seq.Seq.index.? A&. A& s1! i$) (vstd!seq.Seq.index.? A&. A& s2! i$)))) :pattern ((vstd!seq.Seq.index.? A&. A& s1! i$)) :pattern ((vstd!seq.Seq.index.? A&. A& s2! i$)) :qid user_vstd__seq__axiom_seq_ext_equal_14 :skolemid skolem_user_vstd__seq__axiom_seq_ext_equal_14))))) :pattern ((ext_eq false (TYPE%vstd!seq.Seq. A&. A&) s1! s2!)) :qid user_vstd__seq__axiom_seq_ext_equal_15 :skolemid skolem_user_vstd__seq__axiom_seq_ext_equal_15))))
[INFO]

[DEBUG] [edit] proof path: dbg/d6181053ff/proofs/shuffle.1879102912735967004.proof 
[DEBUG] [edit] trace path: dbg/d6181053ff/traces/shuffle.15181022330680520267 
[DEBUG] [skolem] is in the goal! 
[DEBUG] [skolem] i$!skolem_user_lib__spec__cyclicbuffer__log_entry_idx_wrap_around_61!2 -> skv_5ead8fee6076c786_1 
[DEBUG] [edit] proof path: dbg/be920877ca/proofs/shuffle.9555299481395785809.proof 
[DEBUG] [edit] trace path: dbg/be920877ca/traces/shuffle.14575692651422062579 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/d6181053ff/orig.smt2 
[INFO] [edit] [skolem] user_lib__spec__cyclicbuffer__log_entry_idx_wrap_around_61 creates 1 skolem funs, 2 asserts 
(assert (not (=> %%location_label%%0 (forall ((i$ Poly)) (! (=> (has_type i$ INT) (=> (and (and (<= start! (%I i$)) (< (%I i$) (nClip (Add start! buffer_size!)))) (not (= (%I i$) idx!))) (not (= (lib!spec.cyclicbuffer.log_entry_idx.? i$ (I buffer_size!)) (lib!spec.cyclicbuffer.log_entry_idx.? (I idx!) (I buffer_size!)))))) :pattern ((lib!spec.cyclicbuffer.log_entry_idx.? i$ (I buffer_size!))) :qid user_lib__spec__cyclicbuffer__log_entry_idx_wrap_around_61 :skolemid skolem_user_lib__spec__cyclicbuffer__log_entry_idx_wrap_around_61)))))
[INFO] [edit] [banish] user_lib__spec__cyclicbuffer__log_entry_idx_wrap_around_61 
[INFO] [edit] goal was modified! 
[INFO] [edit] written to data/projs/verus.skolem/base.z3/d6181053ff.eb582753.smt2 
7f3387739f00544be2dda14c996725b3.pickle
[WARN] [differ] q

[DEBUG] [skolem] i$!skolem_user_vstd__seq__axiom_seq_ext_equal_9!3 -> skv_e2952d8074bdbf02_1 
[DEBUG] [edit] proof path: dbg/f6f3f962c0/proofs/shuffle.13526046991560484318.proof 
[DEBUG] [edit] trace path: dbg/f6f3f962c0/traces/shuffle.5345052648385214803 


...

[DEBUG] [edit] proof path: dbg/090a2a7d67/proofs/reseed.7014106981689618503.proof 
[DEBUG] [edit] trace path: dbg/090a2a7d67/traces/shuffle.10445631812693282071 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/090a2a7d67/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/090a2a7d67/orig.smt2 
[INFO] [edit] [skolem] user_lib__multilog__inv_v__each_metadata_consistent_with_info_24 creates 1 skolem funs, 1 asserts 
(assert (=> (fuel_bool fuel%lib!multilog.inv_v.each_metadata_consistent_with_info.) (forall ((pm_regions_view! Poly) (multilog_id! Poly) (num_logs! Poly) (cdb! Poly) (infos! Poly)) (! (= (lib!multilog.inv_v.each_metadata_consistent_with_info.? pm_regions_view! multilog_id! num_logs! cdb! infos!) (and (let ((tmp%%$ (vstd!seq.Seq.len.? $ TYPE%lib!multilog.multilogimpl_v.LogInfo. infos!))) (and (and (= (vstd!seq.Seq.len.? $ TYPE%lib!pmem.pmemspec_t.PersistentMemoryRegionView. (Poly%vstd!seq.Seq<lib!pmem.pmemspec_t.PersistentMemoryRegionView.>. (lib!pmem.pmemspec_t.PersistentMemoryRegionsView./PersistentMemoryRegionsView/regions (%Poly%lib!pmem.pmemspec_t.PersistentMemoryRegionsView. pm_regions_view!)))) tmp%%$) (= tmp%%

[DEBUG] [skolem] which_log$!skolem_user_lib__multilog__inv_v__each_metadata_consistent_with_info_24!4 -> skv_a4c05443bf5f6608_1 
[DEBUG] [edit] proof path: dbg/7d8c4302ab/proofs/reseed.9908923975124208297.proof 
[DEBUG] [edit] trace path: dbg/7d8c4302ab/traces/shuffle.15513482486857776214 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/7d8c4302ab/orig.smt2 
[INFO] [edit] [skolem] user_vstd__seq__axiom_seq_ext_equal_9 creates 1 skolem funs, 1 asserts 
(assert (=> (fuel_bool fuel%vstd!seq.axiom_seq_ext_equal.) (forall ((A&. Dcr) (A& Type) (s1! Poly) (s2! Poly)) (! (=> (and (has_type s1! (TYPE%vstd!seq.Seq. A&. A&)) (has_type s2! (TYPE%vstd!seq.Seq. A&. A&))) (= (ext_eq false (TYPE%vstd!seq.Seq. A&. A&) s1! s2!) (and (= (vstd!seq.Seq.len.? A&. A& s1!) (vstd!seq.Seq.len.? A&. A& s2!)) (forall ((i$ Poly)) (! (=> (has_type i$ INT) (=> (and (<= 0 (%I i$)) (< (%I i$) (vstd!seq.Seq.len.? A&. A& s1!))) (= (vstd!seq.Seq.index.? A&. A& s1! i$) (vstd!seq.Seq.index.? A&. A& s2! i$)))) :pattern ((vstd!seq.Seq.index.? A&. A& s1! i$)) :pattern ((vstd!seq.Seq.index.? A&. A& s2! i$)) :qid user_vstd__seq__axiom_seq_ext_equal_9 :skolemid skolem_user_vstd__seq__axiom_seq_ext_equal_9))))) :pattern ((ext_eq false (TYPE%vstd!seq.Seq. A&. A&) s1! s2!)) :qid user_vstd__seq__axiom_

[DEBUG] [skolem] i$!skolem_user_vstd__seq__axiom_seq_ext_equal_9!5 -> skv_23da4d09114d6832_1 


In [17]:
print(skolem_failed)

['data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2']
